In [3]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.art3d as art3d
from scipy.optimize import minimize, differential_evolution, line_search, BFGS, SR1, NonlinearConstraint
from numdifftools import Gradient

In [ ]:
x = np.linspace(-3, 3)
f = lambda x: x ** 2 / 2
slope = lambda x: 2 * x

#Define tangent line
def tan_line(x, x0):
    return slope(x0) * (x - x0) + f(x0)

phi = f(x)

fig, ax = plt.subplots(figsize=[6, 4], dpi=150)

ax.plot(x, phi, color="navy", label=r"$f(x)$")

x0 = np.array([0])
line_x0 = np.array([-2, 2]) + x0
ax.scatter(x0, f(x0), label=r"$f(x*)$", color="black")
ax.plot(line_x0, tan_line(line_x0, x0), color="black", linestyle=":")

ax.legend()

ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$f(x)$")

plt.tight_layout()

plt.show()